In [10]:
from atomicscale_dashboard.artists import PointArtist, ImageArtist, LinesArtist, GraphArtist
from atomicscale_dashboard.canvas import Canvas, TimeLine
from atomicscale_dashboard.extensions import ImageFileLoader, GaussianFilterWidget, ClampWidget
from atomicscale_dashboard.extensions import TEMNetMetadataLoader
from atomicscale_dashboard.extensions import GraphWidget, ItemSelector, DictionaryPrint, IntSliderWithButtons
from atomicscale_dashboard.extensions import link
from atomicscale_dashboard.tools import PanZoom, BoxZoom, AddPoint, DeletePoint, DragPoint, PolygonTool
from atomicscale_dashboard.utils import walk_dir, insert_folder_in_path
from IPython.core.display import display, HTML

import ipywidgets as widgets
from traitlets import directional_link

In [11]:
display(HTML("<style>.container { width:80% !important; }</style>"))

In [12]:
base_path = 'D:\\data\\hBN_export'
fnames = walk_dir(base_path,'.tif')

#fnames

In [13]:
file_selector = ItemSelector(sequence=fnames[:])

image_file_loader = ImageFileLoader()
directional_link((file_selector, 'current_item'), (image_file_loader, 'path'))

metadata_loader = TEMNetMetadataLoader(base_path=base_path)
directional_link((file_selector, 'current_item'), (metadata_loader, 'path'))

frame_selector = ItemSelector(debounce=.1)
directional_link((image_file_loader, 'images'), (frame_selector, 'sequence'))
link((frame_selector, 'current_index'), (metadata_loader, 'current_frame'))

gaussian_filter = GaussianFilterWidget()
directional_link((frame_selector, 'current_item'), (gaussian_filter, 'image_in'))

display_image_artist = ImageArtist()
directional_link((gaussian_filter, 'image_out'), (display_image_artist, 'image'))

point_artist = PointArtist()

link((metadata_loader, 'current_points'), (point_artist, 'points'))
link((metadata_loader, 'current_labels'), (point_artist, 'labels'))

lines_artist = LinesArtist(colors=['lime'])
link((metadata_loader, 'current_segments'), (lines_artist, 'lines'))

graph = GraphWidget()
link((point_artist, 'points'), (graph, 'points'))

graph_artist = LinesArtist()
link((graph, 'edges'), (graph_artist, 'lines'))

canvas = Canvas()

timeline = TimeLine()
link((metadata_loader, 'segment_areas'), (timeline, 'data'))
link((timeline, 'current_index'), (frame_selector, 'current_index'))

canvas.artists = {'Defects':lines_artist, 'Points':point_artist, 'Graph':graph_artist, 'Image':display_image_artist}
canvas.tools = {'Pan': PanZoom(), 
                'Box zoom':BoxZoom(), 
                'Drag point': DragPoint(point_artist=point_artist),
                'Add point B':AddPoint(point_artist=point_artist, label=0), 
                'Add point N':AddPoint(point_artist=point_artist, label=1),
                'Delete point':DeletePoint(point_artist=point_artist, tolerance=50),
                'Add polygon':PolygonTool(lines_artist=lines_artist),
                'Remove polygon':PolygonTool(lines_artist=lines_artist, subtract=True)}

fname_text = widgets.HTML(description='Current file:')

text_area = widgets.Textarea(value='', description='', layout=widgets.Layout(width='400px', height='300px'))
directional_link((metadata_loader, 'summary'), (text_area, 'value'))

summary_text = widgets.Text(value='summaries', description='', layout=widgets.Layout(width='400px'))
write_summary_button = widgets.Button(description='Write summary')
summary_writer = widgets.HBox([summary_text, write_summary_button])

def write_summary(*args):
    path = insert_folder_in_path(base_folder, 'analysis', file_selector.current_item)
    path = os.path.splitext(path)[0] + '.csv'
    
    Path(os.path.split(path)[0]).mkdir(parents=True, exist_ok=True)
    
    with open(path, 'w') as f:
        f.write(text_area.value)

write_summary_button.on_click(write_summary)

right_panel = widgets.VBox([fname_text, 
                            file_selector.widget, 
                            frame_selector.widget, 
                            gaussian_filter.widget, 
                            timeline.widget, 
                            metadata_loader.widget,
                            text_area,
                            summary_writer])

app = widgets.VBox([widgets.HBox([canvas.widget, right_panel])])
app